
<img width="60" src="https://drive.google.com/uc?export=view&id=1JQRWCUpJNAvselJbC_K5xa5mcKl1gBQe"> 



In [0]:
# Uploading files from your local file system

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Import the dataset and verify basic information

- **NAME** - name of school
- **Type** -  provides information about public elementary schools and kindergartens of Natal-RN in 2014.
- **GEOCODE_INPUT** - school's address
- **LAT** - latitute coordinate of school
- **LNG** - longitude coordinate of school




In [0]:
import pandas as pd

# read the dataset and delete the first columns
data = pd.read_csv("edunatal.csv").iloc[:,1:].dropna().reset_index(drop=True)
data.head()

In [0]:
data.info()

## Preparing the data

In [0]:
# install folium packages
!pip install folium geocoder

In [0]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import folium
from folium.plugins import HeatMap
import branca
from folium import plugins
import geocoder

In [0]:
# Calculate the geolocation coordenates 
for i in tqdm(range(len(data))):
    g = geocoder.arcgis(data.loc[i,'GEOCODE_INPUT'])
    data.loc[i,'LAT'] = g.lat
    data.loc[i,'LNG'] = g.lng

In [0]:
data.head()

## Data Analysis 

In [0]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Dictionary for colors
unit_type_colors = {
    'Elem': 'green',
    'KGAR': 'red',
}

# Dictionary for icons
unit_type_icons = {
    'Elem': 'graduation-cap',
    'KGAR': 'odnoklassniki',
}

In [0]:
# Ploting the educational units on map
for i in tqdm(range(len(data))):
    folium.Marker([data.loc[i,'LAT'], data.loc[i,'LNG']],
                  icon=folium.Icon(
                          color = unit_type_colors[data.loc[i,'Type']],
                          icon = unit_type_icons[data.loc[i,'Type']],
                          prefix='fa'),
                  popup = data.loc[i, 'NAME']
        ).add_to(m)

m

In [0]:
coordinates_elem = []
coordinates_kgarten = []

for i in tqdm(range(len(data))):
    if (data.loc[i,'Type'] == 'Elem'):
        coordinates_elem.append([data.loc[i,'LAT'], data.loc[i,'LNG']])
    else:
        coordinates_kgarten.append([data.loc[i,'LAT'], data.loc[i,'LNG']])
        
# Create a map object
m_elem = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Create a map object
m_kgarten = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Create a heat map
HeatMap(coordinates_elem).add_to(m_elem)
HeatMap(coordinates_kgarten).add_to(m_kgarten)


In [0]:
f = branca.element.Figure()

d1 = f.add_subplot(1, 2, 1)
d2 = f.add_subplot(1, 2, 2)

d1.add_child(m_elem)
d2.add_child(m_kgarten)